# Apresentação:

    Devido as demandas por rodar o hysplit para uma quantidade elevada de áreas
    a ideia aqui é conseguir contruir um código para automação desse processo.

# Conectando:

    Precisamos conectar o nosso selenium com o drive que estaremos
    utilizando. Devido a problemas tidos com o chromedriver, utilizaremos
    aqui geckodriver (Mozilla Firefox).

In [300]:
# Importações para automação web:
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.firefox import GeckoDriverManager
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC

# Importações Suporte:
from time import sleep
from tqdm import tqdm

# Imporações para algebra-linear:
import numpy as np
import pandas as pd

# Importações para Salvar o arquivo:
import os
import requests

**Conectando com o navegador**

In [301]:
# Instanciando a classe Service:
service = Service(GeckoDriverManager().install()) # Usada para iniciar uma instância do WebDriver

# Instanciando a classe que define o webdriver:
options = webdriver.FirefoxOptions()

# Intanciando o webdriver:
navegador = webdriver.Firefox(service=service, options=options)

**Acessando a página dos dados de L2 do Calípso**

    Não há a necessidade de se abrir essa página, mas caso
    se queira ir testando por si mesmo a operação do sistema,
    já está a página aqui iniciada.

In [302]:
# Abrindo a aba do 
url = 'https://subset.larc.nasa.gov/calipso/login.php'
navegador.get(url)
sleep(2)

# Abrindo o site:
validacao_login = False
while not validacao_login:
    try:
        navegador.find_element('xpath','//*[@id="login"]').click()
        validacao_login = True  # Defina validacao como True se o elemento for encontrado
        print('Elemento encontrado e clicado com sucesso!')
    except NoSuchElementException:
        sleep(1)  # Espere 1 segundo antes de tentar novamente


# Dados Sensiveis:

with open('dados_cadastrais.txt','r') as arq:
    linhas = arq.readlines()

email = linhas[0].replace('\n','').replace(' ','').split(':')[1]
senha = linhas[1].replace('\n','').replace(' ','').split(':')[1]

# Preenchimento:
sleep(1)
navegador.find_element('xpath','//*[@id="username"]').send_keys(email)
navegador.find_element('xpath','//*[@id="password"]').send_keys(senha)
navegador.find_element('xpath','//*[@id="login"]/p[8]/input').click()

Elemento encontrado e clicado com sucesso!


# Hysplit:

In [303]:
# Faixas que compõem a análise da região atlântico:
atlantico = np.array([[-50,0,10,4],[-45,-3,10,0],[-38,-5,12,-3],[-35,-9,13,-5],
          [-37,-12,14,-9],[-39,-15,14,-12],[-39,-18,12,-15],[-40,-21,13,-18],
          [-42,-23,15,-21],[-48,-25,15,-23],[-49,-27,15,-25],[-49,-29,17,-27],
          [-52,-32,18,-29],[-54,-35,19,-32]])

print('Tamanho da nossa matriz de áreas:',faixas.shape)

Tamanho da nossa matriz de áreas: (14, 4)


In [304]:
# áreas da Bahia:
bahia = np.array([[-40,-15,-37,-11],[-40.4,-10.7,-38,-9.5],[-43.3,-12,-41.3,-9.7],
                  [-42.7,-14.8,-40.8,-12.7],[-45.7,-14,-43.8,-11]])

**Criando pasta, caso ela não exista**

    Caso já haja pasta criada para salvar esses arquivos,
    o proprio código vai reconhece-la e nada será feito.

In [305]:
# Definindo altura:
altura = int(input('Digite a altura: \n')) # Colocar como input como medida de segurança

# Especifique o caminho da pasta que deseja criar
pasta = f"hysplit_trajetorias_{altura}"

# Caminho da pasta
diretorio = r"C:\Users\amori\OneDrive\Área de Trabalho\Análises\Analise - Bahia"

# Juntar pasta ao diretório:
caminho_completo = os.path.join(diretorio, pasta)

# Verifique se a pasta não existe antes de criar
if not os.path.exists(caminho_completo):
    os.makedirs(caminho_completo)
    print("Pasta criada com sucesso!")
else:
    print("Utilizando pasta já existente.")

Digite a altura: 
3000
Pasta criada com sucesso!


**Funções Suporte**

In [306]:
def mod(num):
    '''
        Função construida para
    pegar o modulo de um valor
    qualquer.
    
    Args:
    - float or int: numero que se queira
    tirar o modulo.
    
    Output:
    - float ou int: modulo do valor que
    se tenha dado como entrada.
    '''
    
    # Checando sinal:
    if num <0:
        return -1*num
    
    else:
        return num

def reparticao(left,right,qtd_partes):
    """"
        Função que pega o tamanho total
    da área de divide em áreas menores. Funciona
    apenas para a divisão em 1 dimensão apenas.
    """
    
    # Executando os calculos:
    total = mod(right)+mod(left)
    v = total / qtd_partes
    partes = [left] # Já começa com o valor inicial:
    
    
    # Criando o vetor de respostas:
    for r in range(0, qtd_partes):
        left = left + v 
        partes.append(left)
        
    return partes

In [307]:
# Criando a página do Hysplit:
navegador.execute_script("window.open('https://www.ready.noaa.gov/hypub-bin/trajtype.pl', '_blank');")
sleep(2)

# Mudar o contexto do driver para a nova aba
navegador.switch_to.window(navegador.window_handles[1])
sleep(2)

In [308]:
roi = input('Região de Interesse (atlantico ou bahia):\n')

if roi == 'atlantico':
    # Vamos la...
    for j,faixa in enumerate(atlantico):
        # Calculando as áreas:
        vetor_area = reparticao(faixa[0], faixa[1], 12)
        
        # Criando pasta da faixa:
        pasta_faixa = f'Faixa{j+1}'
            
        # Juntar pasta ao diretório:
        caminho_faixas = os.path.join(caminho_completo, pasta_faixa)
        
        # Verifique se a pasta não existe antes de criar
        if not os.path.exists(caminho_faixas):
            os.makedirs(caminho_faixas)
        else:
            pass
        
        for i in tqdm(range(0, len(vetor_area) - 1)):
            # abrindo o site:
            navegador.get("https://www.ready.noaa.gov/hypub-bin/trajtype.pl")
            
            # Escolhendo o tipo de trajetória:
            validacao_trajetoria = False
            while not validacao_trajetoria:
                try:
                    typeTraj_button = navegador.find_element('xpath','/html/body/div/table/tbody/tr/td/div[6]/form/div/table/tbody/tr[2]/td[2]/input[3]')
                    typeTraj_button.click()
                    validacao_trajetoria = True  # Defina validacao como True se o elemento for encontrado
                except NoSuchElementException:
                    sleep(1)  # Espere 1 segundo antes de tentar novamente
            
            sleep(1)
            # Indo para a proxima página:
            next_button = navegador.find_element('xpath','/html/body/div/table/tbody/tr/td/div[6]/form/center/input')
            next_button.click()
            
            # Selecionando Dominio meteorológico:
            validacao_meteorológico = False
            b = []
            while not validacao_meteorológico and len(b) == 0:
                try:
                    dominio_button = navegador.find_elements('xpath','/html/body/div/table/tbody/tr/td/div[4]/div[2]/form/div/div/table[1]/tbody/tr[1]/td[2]/select')
                    b = dominio_button[0] # Selecione o primeiro elemento da lista
                    validacao_meteorológico = True  # Defina validacao como True se o elemento for encontrado
                except NoSuchElementException:
                    sleep(2)  # Espere 2 segundo antes de tentar novamente
            
            sleep(1)
            
            # Envie a tecla 'g' para o elemento
            b.send_keys('g')
    
                
            # Extraindo Longitude e Latitude:
            lat = round((faixa[1] + faixa[3]) / 2,5)
            long = round((vetor_area[i] + vetor_area[i + 1]) / 2,5)
            
            sleep(1)
            # Achando o elemento:
            DDLat = navegador.find_element('xpath','//*[@id="LatId"]') # Decimal Degrees Latitude
            DDLong = navegador.find_element('xpath','//*[@id="LonId"]') #Decimal Degrees Longitude
            next_button = navegador.find_element('xpath','/html/body/div/table/tbody/tr/td/div[4]/div[2]/form/div/center/input[2]')
            
            sleep(1)
            # Enviando valores:
            DDLat.clear()
            DDLong.clear()
            DDLat.send_keys(str(lat))
            DDLong.send_keys(str(long))
            next_button.click()
            
            sleep(1)
            # Submetendo para a próxima página:
            validacao_submit = False
            while not validacao_submit:
                try:
                    submit = navegador.find_element('xpath','/html/body/div/table/tbody/tr/td/div[4]/form/center/input')
                    validacao_submit = True  # Defina validacao como True se o elemento for encontrado
                    submit.click()
                except NoSuchElementException:
                    sleep(1)  # Espere 1 segundo antes de tentar novamente
            
            sleep(2) 
            
            # Submetendo para a próxima página:
            validacao_traj_direct = False
            while not validacao_traj_direct:
                try:
                    traj_direct = navegador.find_element('xpath','/html/body/div/table/tbody/tr/td/div[4]/form/div[2]/table[1]/tbody/tr[2]/td[2]/input') # Sentido da trajetória
                    validacao_traj_direct = True  # Defina validacao como True se o elemento for encontrado
                    traj_direct.click()
                except NoSuchElementException:
                    sleep(1)  # Espere 1 segundo antes de tentar novamente
            
            DDLat_correction = navegador.find_element('xpath','/html/body/div/table/tbody/tr/td/div[4]/form/div[2]/table[2]/tbody/tr[7]/td[2]/input')
            DDLong_correction = navegador.find_element('xpath','/html/body/div/table/tbody/tr/td/div[4]/form/div[2]/table[2]/tbody/tr[9]/td[2]/input')
            heght = navegador.find_element('xpath','/html/body/div/table/tbody/tr/td/div[4]/form/div[2]/table[2]/tbody/tr[17]/td[2]/input')  # altura level 1
            color = navegador.find_element('xpath','/html/body/div/table/tbody/tr/td/div[4]/form/div[3]/table[2]/tbody/tr[12]/td[2]/input') # Colore o plot das trajetórias
            
            sleep(1)
            # Configurando Elementos - Clica na opção adequada:
            traj_direct.click()
            color.click()
            
            sleep(1)
            # Limpando o entradas:
            heght.clear() 
            DDLat_correction.clear()
            DDLong_correction.clear()
            
            # Inserinado valores:
            heght.send_keys(altura) # Altura média dos nossos aerossois.
            DDLat_correction.send_keys(str(lat))
            DDLong_correction.send_keys(str(long))
            
            
            # Request para a etapa final:
            validacao_request = False
            while not validacao_request:
                try:
                    submit_final = navegador.find_element('xpath','/html/body/div/table/tbody/tr/td/div[4]/form/table/tbody/tr/td/input')
                    submit_final.click()
                    validacao_request = True  # Defina validacao como True se o elemento for encontrado
                except NoSuchElementException:
                    sleep(1)  # Espere 1 segundo antes de tentar novamente
            
            sleep(2)
            # Acessando a URL do plot em PDF:
            validacao_submit_final = False
            while not validacao_submit_final:
                try:
                    pdf = navegador.find_element('xpath','/html/body/div/table/tbody/tr/td/div[4]/table[1]/tbody/tr/td/font/table/tbody/tr[3]/td[2]/font/b/a')
                    validacao_submit_final = True  # Defina validacao como True se o elemento for encontrado
                    # Salvando esse link:
                    link = pdf.get_attribute("href")
            
                    # Lidando com falhas na conexão:
                    operacao_bemsucedida = False
                    while not operacao_bemsucedida:
                        try:
                            # Requerindo resposta:
                            response = requests.get(link)
                            
                            # Baixando o arquivo:
                            if response.status_code == 200:
                                
                                # Criando caminho completo do arquivo:
                                arquivo = f'Trajetoria_faixa_{j+1}_area_{i+1}.pdf'
                                caminho_arquivo = os.path.join(caminho_faixas, arquivo)
                                
                                # Baixa os arquivos:
                                with open(caminho_arquivo, "wb") as file:
                                    file.write(response.content)
                                operacao_bemsucedida = True # Chave de saida do loop
                                
                        # Se não conseguiu acesso, tenta de novo:
                        except requests.exceptions.RequestException as e:
                            print(f"Tentativa {i+1} falhou na faixa {j+1}:", e)
                            sleep(10)
                    
                except NoSuchElementException:
                    # Buscando por erro no Hysplit
                    try:
                        erro = navegador.find_element('xpath','/html/body/div/table/tbody/tr/td/div[4]/table[1]/tbody/tr/td/font')
                        print(f'Erro na área{i+1} da faixa{j+1}:')
                        print(erro.text)
                        continue # Passando para próxima área, essa deu problema:
                    except NoSuchElementException:
                        sleep(2)  # Espere 1 segundo antes de tentar novamente
                        
            sleep(10)

elif roi == 'bahia':          
    # Vamos la...
    for j,faixa in tqdm(enumerate(bahia)):      
        # Criando pasta da faixa:
        pasta_faixa = f'Bahia_Trajetórias_{altura}'
            
        # Juntar pasta ao diretório:
        caminho_faixas = os.path.join(caminho_completo, pasta_faixa)
        
        # Verifique se a pasta não existe antes de criar
        if not os.path.exists(caminho_faixas):
            os.makedirs(caminho_faixas)
        else:
            pass
        
        # abrindo o site:
        navegador.get("https://www.ready.noaa.gov/hypub-bin/trajtype.pl")
        
        # Escolhendo o tipo de trajetória:
        validacao_trajetoria = False
        while not validacao_trajetoria:
            try:
                typeTraj_button = navegador.find_element('xpath','/html/body/div/table/tbody/tr/td/div[6]/form/div/table/tbody/tr[2]/td[2]/input[3]')
                typeTraj_button.click()
                validacao_trajetoria = True  # Defina validacao como True se o elemento for encontrado
            except NoSuchElementException:
                sleep(1)  # Espere 1 segundo antes de tentar novamente
        
        sleep(1)
        # Indo para a proxima página:
        next_button = navegador.find_element('xpath','/html/body/div/table/tbody/tr/td/div[6]/form/center/input')
        next_button.click()
        
        # Selecionando Dominio meteorológico:
        validacao_meteorológico = False
        b = []
        while not validacao_meteorológico and len(b) == 0:
            try:
                dominio_button = navegador.find_elements('xpath','/html/body/div/table/tbody/tr/td/div[4]/div[2]/form/div/div/table[1]/tbody/tr[1]/td[2]/select')
                b = dominio_button[0] # Selecione o primeiro elemento da lista
                validacao_meteorológico = True  # Defina validacao como True se o elemento for encontrado
            except NoSuchElementException:
                sleep(2)  # Espere 2 segundo antes de tentar novamente
        
        sleep(1)
        
        # Envie a tecla 'g' para o elemento
        b.send_keys('g')
    
            
        # Calculando coordenada central da imagem:
        lat = round((faixa[1] + faixa[3]) / 2,5)
        long = round((faixa[0] + faixa[2]) / 2,5)
        
        sleep(1)
        # Achando o elemento:
        DDLat = navegador.find_element('xpath','//*[@id="LatId"]') # Decimal Degrees Latitude
        DDLong = navegador.find_element('xpath','//*[@id="LonId"]') #Decimal Degrees Longitude
        next_button = navegador.find_element('xpath','/html/body/div/table/tbody/tr/td/div[4]/div[2]/form/div/center/input[2]')
        
        sleep(1)
        # Enviando valores:
        DDLat.clear()
        DDLong.clear()
        DDLat.send_keys(str(lat))
        DDLong.send_keys(str(long))
        next_button.click()
        
        sleep(1)
        # Submetendo para a próxima página:
        validacao_submit = False
        while not validacao_submit:
            try:
                submit = navegador.find_element('xpath','/html/body/div/table/tbody/tr/td/div[4]/form/center/input')
                validacao_submit = True  # Defina validacao como True se o elemento for encontrado
                submit.click()
            except NoSuchElementException:
                sleep(1)  # Espere 1 segundo antes de tentar novamente
        
        sleep(2) 
        
        # Submetendo para a próxima página:
        validacao_traj_direct = False
        while not validacao_traj_direct:
            try:
                traj_direct = navegador.find_element('xpath','/html/body/div/table/tbody/tr/td/div[4]/form/div[2]/table[1]/tbody/tr[2]/td[2]/input') # Sentido da trajetória
                validacao_traj_direct = True  # Defina validacao como True se o elemento for encontrado
                traj_direct.click()
            except NoSuchElementException:
                sleep(1)  # Espere 1 segundo antes de tentar novamente
        
        DDLat_correction = navegador.find_element('xpath','/html/body/div/table/tbody/tr/td/div[4]/form/div[2]/table[2]/tbody/tr[7]/td[2]/input')
        DDLong_correction = navegador.find_element('xpath','/html/body/div/table/tbody/tr/td/div[4]/form/div[2]/table[2]/tbody/tr[9]/td[2]/input')
        heght = navegador.find_element('xpath','/html/body/div/table/tbody/tr/td/div[4]/form/div[2]/table[2]/tbody/tr[17]/td[2]/input')  # altura level 1
        color = navegador.find_element('xpath','/html/body/div/table/tbody/tr/td/div[4]/form/div[3]/table[2]/tbody/tr[12]/td[2]/input') # Colore o plot das trajetórias
        
        sleep(1)
        # Configurando Elementos - Clica na opção adequada:
        traj_direct.click()
        color.click()
        
        sleep(1)
        # Limpando o entradas:
        heght.clear() 
        DDLat_correction.clear()
        DDLong_correction.clear()
        
        # Inserinado valores:
        heght.send_keys(altura) # Altura média dos nossos aerossois.
        DDLat_correction.send_keys(str(lat))
        DDLong_correction.send_keys(str(long))
        
        
        # Request para a etapa final:
        validacao_request = False
        while not validacao_request:
            try:
                submit_final = navegador.find_element('xpath','/html/body/div/table/tbody/tr/td/div[4]/form/table/tbody/tr/td/input')
                submit_final.click()
                validacao_request = True  # Defina validacao como True se o elemento for encontrado
            except NoSuchElementException:
                sleep(1)  # Espere 1 segundo antes de tentar novamente
        
        sleep(10)
        # Acessando a URL do plot em PDF:
        validacao_submit_final = False
        while not validacao_submit_final:
            try:
                pdf = navegador.find_element('xpath','/html/body/div/table/tbody/tr/td/div[4]/table[1]/tbody/tr/td/font/table/tbody/tr[3]/td[2]/font/b/a')
                validacao_submit_final = True  # Defina validacao como True se o elemento for encontrado
                # Salvando esse link:
                link = pdf.get_attribute("href")
        
                # Lidando com falhas na conexão:
                operacao_bemsucedida = False
                while not operacao_bemsucedida:
                    try:
                        # Requerindo resposta:
                        response = requests.get(link)
                        
                        # Baixando o arquivo:
                        if response.status_code == 200:
                            
                            # Criando caminho completo do arquivo:
                            arquivo = f'Trajetoria_area_{j+1}.pdf'
                            caminho_arquivo = os.path.join(caminho_faixas, arquivo)
                            
                            # Baixa os arquivos:
                            with open(caminho_arquivo, "wb") as file:
                                file.write(response.content)
                            operacao_bemsucedida = True # Chave de saida do loop
                            
                    # Se não conseguiu acesso, tenta de novo:
                    except requests.exceptions.RequestException as e:
                        print(f"Tentativa {i+1} falhou na faixa {j+1}:", e)
                        sleep(10)
                
            except NoSuchElementException:
                # Buscando por erro no Hysplit
                try:
                    erro = navegador.find_element('xpath','/html/body/div/table/tbody/tr/td/div[4]/table[1]/tbody/tr/td/font')
                    print(f'Erro na área{i+1} da faixa{j+1}:')
                    print(erro.text)
                    continue # Passando para próxima área, essa deu problema:
                except NoSuchElementException:
                    sleep(2)  # Espere 1 segundo antes de tentar novamente
                    
        sleep(10)
        
    print('Acabou os donwloads')

Região de Interesse (atlantico ou bahia):
bahia
Acabou os donwloads


In [310]:
# Encerrando o procedimento:
navegador.quit()